In [2]:
!pip install langchain_community faiss-gpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 924.9 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 17.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling

In [3]:
from langchain_community.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_community.document_loaders.csv_loader import CSVLoader
#from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain_community.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM
)

import sys, csv
csv.field_size_limit(sys.maxsize)

131072

In [4]:
loader = CSVLoader(
        file_path="/kaggle/input/refomed/test_kb_petite.csv", source_column="title",
        csv_args={
            "delimiter": "\t",
            #"quotechar": '"',
            "fieldnames": ["index", "title", "text", "length"],
            },
        )
page_content_column = "text"
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

# 'data' holds the text you want to split, splitthe text into documents using the text splitter.
docs = text_splitter.split_documents(data)

modelPath = "Lajavaness/sentence-camembert-large"
#modelPath = "sentence-transformers/all-MiniLM-L6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
    )

db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 3})

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.79k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/809k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [6]:
question = "Expliquez-moi, en termes simples ou paraphrase ou définition : hémoglobinopathie?" #hémoglobinopathie?"

docs = retriever.get_relevant_documents(question)
print(docs)

[Document(page_content="index: 80856\ntitle: hémoglobine\ntext: Structure et fonctionnement = Sous-unités =L'hémoglobine possède une structure quaternaire caractéristique de nombreuses protéines à sous-unités globulaires.\nlength: 17", metadata={'source': 'hémoglobine', 'row': 8541}), Document(page_content="index: 80837\ntitle: hémoglobine\ntext: Hémoglobine https://fr.wikipedia.org/wiki/H%C3%A9moglobine L'hémoglobine, couramment symbolisée Hb, parfois Hgb, est un pigment respiratoire (de la famille moléculaire des métalloprotéines, ici contenant du fer) présent essentiellement dans le sang des vertébrés, au sein de leurs globules rouges, ainsi que dans les tissus de certains invertébrés.\nlength: 43", metadata={'source': 'hémoglobine', 'row': 8523}), Document(page_content="index: 80858\ntitle: hémoglobine\ntext: Les sections hélicoïdales sont stabilisées par des liaisons hydrogène qui confèrent à la protéine sa structure tridimensionnelle caractéristique, appelée repliement globine ca

In [12]:
import re
def remove_urls(text):
    # Regex pattern to match http and https URLs
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    # Substitute matched URLs with an empty string
    cleaned_text = re.sub(url_pattern, '', text)
    return cleaned_text


def prepare_context(docs):
    cleaned_context = []
    refs = []
    for d in docs:
        refs.append(str(d.metadata['row']))
        raw_d = d.page_content.split('text:')[-1]
        cleaned_d = remove_urls(raw_d)
        #print(raw_d ,"|", cleaned_d.split('length:')[0])
        cleaned_context.append(cleaned_d.split('length:')[0])

    return ' '.join(cleaned_context).strip().replace('\n', ''), '-'.join(refs)

prepare_context(docs)

("Structure et fonctionnement = Sous-unités =L'hémoglobine possède une structure quaternaire caractéristique de nombreuses protéines à sous-unités globulaires.  Hémoglobine  L'hémoglobine, couramment symbolisée Hb, parfois Hgb, est un pigment respiratoire (de la famille moléculaire des métalloprotéines, ici contenant du fer) présent essentiellement dans le sang des vertébrés, au sein de leurs globules rouges, ainsi que dans les tissus de certains invertébrés.  Les sections hélicoïdales sont stabilisées par des liaisons hydrogène qui confèrent à la protéine sa structure tridimensionnelle caractéristique, appelée repliement globine car on le retrouve également dans d'autres globines à groupe prosthétique héminique telles que la myoglobine.",
 '8541-8523-8543')

In [7]:
import pandas as pd
from tqdm import tqdm
import time

test_df = pd.read_csv('/kaggle/input/refomed/refomed_test.csv', sep='\t')
val_df = pd.read_csv('/kaggle/input/refomed/refomed_val.csv', sep='\t')

plm2key = {
           "moussaKam/mbarthez":"mbarthez",
           "moussaKam/barthez":"barthez",
            "moussaKam/barthez-orangesum-abstract": "barthez-orangesum",
           "moussaKam/mbarthez-dialogue-summarization":"mbarthez-dialogue",
          }



In [19]:
device = 'cuda' #torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for plm in list(plm2key.keys()):
    
    barthez_tokenizer = AutoTokenizer.from_pretrained(plm)
    barthez_model = AutoModelForSeq2SeqLM.from_pretrained(plm).to(device)
    
    for ip,prompt_template in enumerate(["Expliquez-moi, en termes simples ou paraphrase ou définition :",
                "Expliquez-moi le terme médical en mots simples, par une paraphrase ou une courte définition :",
              ]):
        
        print("running", plm,"for p",ip+3)

        for T in [25, 50]:
            
            queries2 = []
            responses2 = []
            tt2 = []
            refs = []
            for k in tqdm(test_df.term):
                
                queries2.append(prompt_template+f' {k}')
                retrieved_docs = retriever.get_relevant_documents(prompt_template+f' {k}')
                cleaned_context,r = prepare_context(retrieved_docs)
                refs.append(r)
                #context_collector.append(cleaned_context)
                #print(k, cleaned_context)
                t1 = time.time()
                input_ids = torch.tensor(
                    [barthez_tokenizer.encode(cleaned_context, add_special_tokens=True)]
                )
                #print(input_ids.shape)
                predict = barthez_model.generate(input_ids.to(device), max_new_tokens=T, num_beams=4, early_stopping=True)[0]
                answer = barthez_tokenizer.decode(predict, skip_special_tokens=True)
                t2 = time.time()
                
                responses2.append(answer)
                tt2.append(t2-t1)
                
            pd.DataFrame({"prompts":queries2, "raga": responses2, "time_taken":tt2, "src_doc":refs
                         }).to_csv(f'final_camem_{plm2key[plm]}_test_t{T}_p{ip+3}.csv', sep='\t', index=False)
            # ================================ dev datframe =====================================


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


running moussaKam/barthez-orangesum-abstract for p 3


  0%|          | 0/1063 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
  0%|          | 1/1063 [00:00<03:59,  4.44it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `earl

running moussaKam/barthez-orangesum-abstract for p 4


  0%|          | 0/1063 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
  0%|          | 1/1063 [00:00<05:05,  3.47it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `earl

...

pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

running moussaKam/barthez for p 3


100%|██████████| 1063/1063 [08:07<00:00,  2.18it/s]


running moussaKam/barthez for p 4


100%|██████████| 1063/1063 [08:39<00:00,  2.04it/s]


In [21]:
!zip -r dev_files.zip /kaggle/working

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)
  adding: kaggle/working/final_camem_mbarthez-dialogue_val_t25_p3.csv (deflated 78%)
  adding: kaggle/working/final_camem_barthez-orangesum_val_t25_p3.csv (deflated 77%)
  adding: kaggle/working/final_camem_mbarthez_val_t50_p3.csv (deflated 76%)
  adding: kaggle/working/final_camem_mbarthez_val_t25_p3.csv (deflated 77%)
  adding: kaggle/working/final_camem_barthez_val_t50_p3.csv (deflated 75%)
  adding: kaggle/working/final_camem_barthez-orangesum_val_t25_p0.csv (deflated 78%)
  adding: kaggle/working/final_camem_barthez_val_t25_p4.csv (deflated 82%)
  adding: kaggle/working/final_camem_barthez-orangesum_val_t25_p4.csv (deflated 82%)
  adding: kaggle/working/final_camem_barthez_val_t50_p4.csv (deflated 80%)
  adding: kaggle/working/final_camem_mbarthez-dialogue_val_t25_p4.csv (deflated 82%)
  adding: kaggle/working/final_camem_mbarthez_val_t50_p4.csv (deflated 80%)
  adding: kaggle/working/fi

In [15]:
responses2

["Le trouble de déficit de l'attention (DDA) est souvent associé à une maladresse ou à une maladresse intelle",
 "Elle peut évoluer d'un seul tenant ou par cycles d'aggravation puis de récupération (poussées).",
 'Sémantique Le mot « myélite » désigne de manière non spécifique toute inflammation de la moelle spinale.',
 "Il s'agit d'une anomalie de la natrémie qui touche presque un tiers des patients hospitalisés.",
 'Cette thèse est réfutée par les historiens spécialistes de Napoléon Ier.',
 'CHRONOLOGIE. Les patients atteints de maladies rares peuvent être atteints de formes graves comme des formes graves de la maladie',
 "Les paralysies peuvent être d'origine nucléaire (atteinte des noyaux oculomoteurs), tronculaires (atteinte du",
 'La vitamine B1, ou thiamine, est un cofacteur présent chez tous les êtres vivants pour y cataly',
 "Selon les spécialistes, cet état d'effondrement représenterait un niveau plus habituel que celui durant lequel existent des États et des",
 'La scoliose 

# Rough

In [32]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# Load the model and tokenizer from Hugging Face
model_name = "moussaKam/barthez"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Create a text generation pipeline
text_generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer,
                         max_new_tokens=500)

prompt_template_fr = """[INST] Utilisez les informations suivantes pour répondre à 
la question de l'utilisateur par une paraphrase,
une explication ou une courte définition.
Contexte : {context}

Expliquez-moi, en termes simples ou paraphrase ou définition : {question}. [/INST]"""

prompt = PromptTemplate(template=prompt_template_fr, input_variables=['context','question'])
#chain_type_kwargs= {"prompt": prompt}

hf = HuggingFacePipeline(pipeline=text_generator)

chain = prompt | hf

question = "hémoglobinopathie"

print(chain.invoke({"context" : ' '.join([d.page_content.split(':')[-2][:-7] for d in docs]),
    "question": question}))


In [36]:
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

import torch

generator = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("jpacifico/French-Alpaca-Phi-3-mini-128k-instruct-v1.0")


READER_LLM = pipeline(
    model=generator,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=50,
)

READER_LLM("c'est quoi cancer?")

from transformers import pipeline
task = "text2text-generation"
model_name = "google/flan-t5-base"
input_text = "Can you answer this question in French: What is your name? based on the context: my name is Piere"
text2text_generator = pipeline(
    task,
    model = model_name)

text2text_generator(input_text)

from transformers import pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

generator = pipeline('text-generation', 
                     model='lightonai/pagnol-large', 
                     trust_remote_code=True,
                    max_length=500)

output = generator(
    "Salut PAGnol, comment ça va ?",
    max_length=500,
    do_sample=True,
    temperature=0.7,
)[0]["generated_text"]
output



from langchain_core.prompts import PromptTemplate
hf = HuggingFacePipeline(pipeline=text2text_generator)

template = """[INST] Question: {question}
Answer: [\INST]"""

prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "c'est la quoi cancer?"

print(chain.invoke({"question": question}))

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a cutaneous cancer
